In [ ]:
# Create folder
%mkdir download
%mkdir data

%mkdir data/train
%mkdir data/val
%mkdir data/test

%mkdir data/train/labels
%mkdir data/val/labels
%mkdir data/test/labels

In [ ]:
# Download and extract

%cd download

# import os
# urls = {
#     'train_dataset.zip': 'https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Training_Input.zip',
#     'train_gt1.zip': 'https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Training_GroundTruth.zip',
    
#     'val_dataset.zip': 'https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Validation_Input.zip',
#     'val_gt1.zip': 'https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Validation_GroundTruth.zip',
    
#     'test_dataset.zip': 'https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Test_Input.zip',
#     'test_gt1.zip': 'https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Test_GroundTruth.zip', 
# }
# for filename, url in urls.items():
#     os.system(f'curl -Lo {filename} {url}')
#     os.system(f'unzip -q {filename}')

!gdown 1-5G5YnR5sAXhxoYNoIxdM7iWveGHTMj9
!gdown 1-9FFXcwLwHJac6mpF2jhdxHTZFzLvVYA
!gdown 1VHMH4E41nDOieOGXbQ_eu13JfqXRDfZH
!gdown 1--HcNLfuBpxavAGHfCiPSR_XRXCV5NzO
!gdown 1-4i12Q7S9NKgitkW9Oy54WW1qOLOPN3R
!gdown 1JV2kD2iV5o3DTjQykDUF3PW0o5gZ5YJt

!mv ISIC2018_Task1-2_Training_Input/ ../data/train/
!mv ISIC2018_Task1-2_Validation_Input/ ../data/val/
!mv ISIC2018_Task1-2_Test_Input/ ../data/test/

!mv ISIC2018_Task1_Training_GroundTruth/ ../data/train/labels/
!mv ISIC2018_Task1_Validation_GroundTruth/ ../data/val/labels/
!mv ISIC2018_Task1_Test_GroundTruth/ ../data/test/labels/

%cd ../data
!mv train/ISIC2018_Task1-2_Training_Input/ train/images
!mv val/ISIC2018_Task1-2_Validation_Input/ val/images
!mv test/ISIC2018_Task1-2_Test_Input/ test/images

!mv train/labels/ISIC2018_Task1_Training_GroundTruth train/labels/task1
!mv val/labels/ISIC2018_Task1_Validation_GroundTruth val/labels/task1
!mv test/labels/ISIC2018_Task1_Test_GroundTruth test/labels/task1
%cd ..

In [ ]:
# create proceeded_data
%mkdir proceeded_data
%mkdir proceeded_data/train
%mkdir proceeded_data/val
%mkdir proceeded_data/test

!python Datasets/process_resize.py

In [ ]:
!git clone https://github.com/yassouali/CCT.git
!git clone https://github.com/Britefury/cutmix-semisup-seg.git
!git clone https://github.com/charlesCXK/TorchSemiSeg.git
!git clone https://github.com/googleinterns/wss.git
!git clone https://github.com/LiheYoung/ST-PlusPlus.git
!git clone https://github.com/Haochen-Wang409/U2PL.git
!git clone https://github.com/Jin-Ying/GTA-Seg.git
!git clone https://github.com/xiaoyao3302/CCVC.git
!git clone https://github.com/LiheYoung/UniMatch.git

In [ ]:
!git clone https://github.com/siyi-wind/AViT.git

In [ ]:
!python -u fixmatch.py --gpu 0 --batch_size 42 --seed 1 --exp fixmatch1

In [ ]:
import os
from IPython.display import display
import matplotlib.pyplot as plt
from PIL import Image
from Datasets.transform import color_jitter
from Datasets.create_dataset import norm01
import numpy as np

train = os.listdir('proceeded_data/train/Image/')
path = 'proceeded_data/train/Image/' + train[3]
img = np.load(path)
plt.imshow(img)
plt.show()

In [ ]:
import albumentations as A
img_size = 224
p = 0.5
aug_transf = A.Compose([
        A.Resize(img_size, img_size),
        A.HorizontalFlip(p=p),
        A.VerticalFlip(p=p)
])
img = aug_transf(image=img.astype('uint8'))['image']
img = norm01(img)
plt.imshow(img)
plt.show()

In [ ]:
import random
import torch
import random
import numpy as np
from torchvision import transforms
from Datasets.transform import *
print(img)
img_s = torch.from_numpy(img).float()
img_s = img_s.permute(2, 0, 1)
print(img_s.shape)
img_s = transforms.ToPILImage()(img_s)
if random.random() < 0.8:
    img_s = transforms.ColorJitter(0.5, 0.5, 0.5, 0.25)(img_s)
img_s = transforms.RandomGrayscale(p=0.2)(img_s)
img_s = blur(img_s, p=0.5)
img_s = transforms.ToTensor()(img_s)
print(img_s)
img_s = img_s.permute(1, 2, 0)
plt.imshow(img_s.numpy())
plt.show()

In [ ]:
from Models.Transformer.SwinUnet import SwinUnet
import torch
from Datasets.create_dataset import SkinDataset
import torchvision.transforms as transforms
import glob
import os
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt

gpu = 0
test_path = './data/test/images/'
result_path = './data/test/labels/task1'
vis_path = './visualize/'
test_data = os.listdir('./data/test/images/')
model_dir = './results/fixmatch2_20240416_0004/best.pth'

trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
resize = transforms.Resize((224, 224))

tensor_to_pil = transforms.ToPILImage()

model = SwinUnet(img_size=224)
model = model.cuda(gpu)
model.load_state_dict(torch.load(model_dir))



In [ ]:
for name in test_data[:10]:
    root_path = os.path.join(test_path, name)
    img = Image.open(root_path).convert('RGB')
    display(resize(img))
    img = trans(img).cuda(gpu).float()
    img = img.unsqueeze(0)
    output = model(img)
    print(output)
    output = torch.sigmoid(output)
    print(output)
    mask = (output > 0.8).float()
    output = output * mask
    print(output)
    output = torch.round(output)
    print(output)
    output = output.squeeze(0)
    img = tensor_to_pil(output)
    result = os.path.join(result_path, name)
    
    display(img)
    plt.show()
    break

In [ ]:
import os
import json
import torch
import random
import numpy as np
from torchvision import transforms
import albumentations as A
import pandas as pd
from Datasets.transform import *
from Datasets.create_dataset import *
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt

# def norm01(x):
#     return np.clip(x, 0, 255) / 255

# class StrongWeakAugment(torch.utils.data.Dataset):
#     def __init__(self, dataset, img_size,  
#         data_path='./proceeded_data/train/'):
#         super(StrongWeakAugment, self).__init__()
        
#         self.dataset = dataset
#         self.root_dir = data_path

#         self.num_samples = len(self.dataset)

#         p = 0.5
#         self.aug_transf = A.Compose([
#             A.Resize(img_size, img_size),
#             A.HorizontalFlip(p=p),
#             A.VerticalFlip(p=p)
#         ])
#         self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                               std=[0.229, 0.224, 0.225])
    

#     def __getitem__(self, index):
#         if torch.is_tensor(index):
#             index = index.tolist()
#         sample_name = self.dataset[index]
#         img_path = self.root_dir + f'Image/{sample_name}'

#         img_data = np.load(img_path)

#         tsf = self.aug_transf(image=img_data.astype('uint8'))
#         img_data = tsf['image']
        
#         img_data = norm01(img_data)

#         img_s = torch.from_numpy(img_data).float()
#         img_w = torch.from_numpy(img_data).float()
#         img_s = img_s.permute(2, 0, 1)
#         img_w = img_w.permute(2, 0, 1)
        
#         img_s = transforms.ToPILImage()(img_s)
#         if random.random() < 0.8:
#             img_s = transforms.ColorJitter(0.5, 0.5, 0.5, 0.25)(img_s)
#         img_s = transforms.RandomGrayscale(p=0.2)(img_s)
#         img_s = blur(img_s, p=0.5)
#         img_s = transforms.PILToTensor()(img_s)

#         return{
#             'img_w': img_w,
#             'img_s': img_s,
#         }

# class SkinDataset(torch.utils.data.Dataset):
#     def __init__(self, dataset, img_size, use_aug=False,  
#         data_path='./proceeded_data/train/'):
#         super(SkinDataset, self).__init__()
        
#         self.dataset = dataset
#         self.root_dir = data_path
#         self.use_aug = use_aug

#         self.num_samples = len(self.dataset)

#         p = 0.5
#         self.aug_transf = A.Compose([
#             A.Resize(img_size, img_size),
#             A.HorizontalFlip(p=p),
#             A.VerticalFlip(p=p),
#             A.GaussNoise(p=p),
#             A.ShiftScaleRotate(p=p),
#             A.RandomBrightnessContrast(p=p),
#         ])
#         self.transf = A.Compose([
#             A.Resize(img_size, img_size),
#         ])
#         self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                               std=[0.229, 0.224, 0.225])
    

#     def __getitem__(self, index):
#         if torch.is_tensor(index):
#             index = index.tolist()
#         sample_name = self.dataset[index]
#         img_path = self.root_dir + f'Image/{sample_name}'
#         label_path = self.root_dir + f'Label/{sample_name}'

#         img_data = np.load(img_path)
#         label_data = np.load(label_path) > 0.5

#         if self.use_aug:
#             tsf = self.aug_transf(image=img_data.astype('uint8'), mask=label_data.astype('uint8'))
#         else:
#             tsf = self.transf(image=img_data.astype('uint8'), mask=label_data.astype('uint8'))
#         img_data, label_data = tsf['image'], tsf['mask']
        
#         img_data = norm01(img_data)
#         label_data = np.expand_dims(label_data, 0)

#         img_data = torch.from_numpy(img_data).float()
#         label_data = torch.from_numpy(label_data).float()

#         img_data = img_data.permute(2, 0, 1)
#         img_data = self.normalize(img_data)


#         return{
#             'image': img_data,
#             'label': label_data,
#         }


#     def __len__(self):
#         return self.num_samples

path = './proceeded_data/train/'
img_size=224
train_data = os.listdir(path + '/Image/')
print(len(train_data))
idx = random.randint(0, len(train_data))
# u_dataset = StrongWeakAugment(dataset=train_data, img_size=img_size, data_path=path)
# tensor_to_pil = transforms.ToPILImage()
# imgs = u_dataset[idx]
# display(tensor_to_pil(imgs['img_w']))
# display(tensor_to_pil(imgs['img_s']))
l_dataset = SkinDataset2(dataset=train_data, img_size=img_size, use_aug=False, data_path=path)
tensor_to_pil = transforms.ToPILImage()
imgs = l_dataset[idx]
# print(imgs['img_w'].shape)
# print(imgs['img_s'].shape)
# display(tensor_to_pil(imgs['img_w']))
# display(tensor_to_pil(imgs['img_s']))
print(imgs['image'].shape)
print(imgs['label'].shape)
display(transforms.ToPILImage()(imgs['image']))
display(transforms.ToPILImage()(imgs['label']))
plt.show()

In [ ]:
import torch
a = torch.tensor([0.9327123912])
torch.round(a)

In [ ]:
from Models.Transformer.SwinUnet import SwinUnet
import torch
import torchvision.transforms as transforms
import glob
import os
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import random
import torch.utils.data
import albumentations as A

random.seed(1)

class SkinDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, img_size, use_aug=False,  
        data_path='./proceeded_data/train/'):
        super(SkinDataset, self).__init__()
        
        self.dataset = dataset
        self.root_dir = data_path

        self.num_samples = len(self.dataset)

        self.augment_img = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])
        ])
        self.augment_label = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor()
        ])
        
    

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        sample_name = self.dataset[index].replace('.jpg', '')
        img_path = self.root_dir + f'images/{sample_name}.jpg'
        label_path = self.root_dir + f'labels/task1/{sample_name}_segmentation.png'

        img_data = Image.open(img_path)
        label_data = Image.open(label_path)
        
        img_data = self.augment_img(img_data)
        label_data = self.augment_label(label_data)


        return{
            'image': img_data,
            'label': label_data,
            'name': self.dataset[index],
        }


    def __len__(self):
        return self.num_samples


gpu = 0
test_path = './data/test/'
result_path = './data/test/labels/task1'
vis_path = './visualize/'
test_data = [name for name in os.listdir(os.path.join(test_path, 'images')) if name.find('.txt') == -1]
model_dir = './results/'
model_folder = 'sup1_20240417_0931'
n_sample = 100
img_size=224

tensor_to_pil = transforms.ToPILImage()

model = SwinUnet(img_size=224)
model = model.cuda(gpu)
full_path_model = os.path.join(model_dir, model_folder, 'best.pth')
model.load_state_dict(torch.load(os.path.join(full_path_model)))

path_vis = os.path.join(vis_path, model_folder)
if os.path.exists(path_vis) == False:
    os.system(f'mkdir {path_vis}')
else:
    os.system(f'rm {path_vis}/*')

samples = random.sample(test_data, n_sample)
test_dataset = SkinDataset(dataset=test_data, img_size=img_size, use_aug=False, data_path=test_path)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                            batch_size=42,
                                            shuffle=False,
                                            num_workers=0,
                                            pin_memory=True,
                                            drop_last=False)

for batch_id, batch in enumerate(test_loader):
    imgs = batch['image'].cuda().float()
    labels = batch['label'].cuda().float()
            
    with torch.no_grad():
                
        outputs = model(imgs)

        outputs = torch.sigmoid(outputs)

        for idx in range(len(outputs)):
            pil_img = tensor_to_pil(outputs[idx])
            pil_img.save(os.path.join(path_vis, batch['name'][idx]))
    
    print('Done')
        

In [ ]:
import os
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import random
random.seed(1)

vis_path = './visualize/'
model_folder = ['sup1_20240417_0931', 'fixmatch1_20240417_1859']
test_folder = 'data/test/labels/task1'

full_path_vis = os.path.join(vis_path, model_folder[-1])

dataset = random.sample(os.listdir(full_path_vis), 100)

for data in dataset:
    image = Image.new('RGB', size=(224 * 3, 224))
    name = data.replace('.jpg', '')
    path_in_test = os.path.join(test_folder, f'{name}_segmentation.png')
    img_test = Image.open(path_in_test).resize((224, 224))
    image.paste(img_test, (0, 0))
    for i in range(len(model_folder)):
        full_path_vis = os.path.join(vis_path, model_folder[i]) 
        path_in_vis = os.path.join(full_path_vis, f'{name}.jpg')
        img_vis = Image.open(path_in_vis)
        image.paste(img_vis, ((i + 1) * 224, 0))
    
    display(image)
plt.show()


In [ ]:
!python -u cct.py --gpu 0 --seed 1 --exp cct1

In [ ]:
!python -u cps.py --gpu 0 --seed 1 --exp cps1

In [ ]:
!python -u cct8.py --gpu 0 --seed 1 --exp cct1

In [ ]:
import torch
import torch.nn as nn
import torch.utils.checkpoint as checkpoint
from einops import rearrange
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
import copy

from torch.distributions.uniform import Uniform
import numpy as np

import torch.nn.functional as F
import random
import cv2


def guided_cutout(output, resize, erase=0.4, use_dropout=False):
    if len(output.shape) == 3:
        masks = (output > 0).float()
    else:
        masks = (output.argmax(1) > 0).float()

    if use_dropout:
        p_drop = random.randint(3, 6)/10
        maskdroped = (F.dropout(masks, p_drop) > 0).float()
        maskdroped = maskdroped + (1 - masks)
        maskdroped.unsqueeze_(0)
        maskdroped = F.interpolate(maskdroped, size=resize, mode='nearest')

    masks_np = []
    for mask in masks:
        mask_np = np.uint8(mask.cpu().numpy())
        mask_ones = np.ones_like(mask_np)
        try: # Version 3.x
            _, contours, _ = cv2.findContours(mask_np, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        except: # Version 4.x
            contours, _ = cv2.findContours(mask_np, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        polys = [c.reshape(c.shape[0], c.shape[-1]) for c in contours if c.shape[0] > 50]
        for poly in polys:
            min_w, max_w = poly[:, 0].min(), poly[:, 0].max()
            min_h, max_h = poly[:, 1].min(), poly[:, 1].max()
            bb_w, bb_h = max_w-min_w, max_h-min_h
            rnd_start_w = random.randint(0, int(bb_w*(1-erase)))
            rnd_start_h = random.randint(0, int(bb_h*(1-erase)))
            h_start, h_end = min_h+rnd_start_h, min_h+rnd_start_h+int(bb_h*erase)
            w_start, w_end = min_w+rnd_start_w, min_w+rnd_start_w+int(bb_w*erase)
            mask_ones[h_start:h_end, w_start:w_end] = 0
        masks_np.append(mask_ones)
    masks_np = np.stack(masks_np)
    print(mask_np.shape)

    maskcut = torch.from_numpy(masks_np).float().unsqueeze_(1)
    print(maskcut.shape)
    maskcut = F.interpolate(maskcut, size=resize, mode='nearest')
    print(maskcut.shape)

    if use_dropout:
        return maskcut.to(output.device), maskdroped.to(output.device)
    return maskcut.to(output.device)


class CutOut(nn.Module):
    def __init__(self, erase=0.4):
        super(CutOut, self).__init__()
        self.erase = erase 

    def forward(self, x, pred=None):
        maskcut = guided_cutout(pred, erase=self.erase, resize=(x.size(2)))
        print(x.shape, maskcut.shape)
        x = x * maskcut
        return x
    
feature = torch.rand((8, 3136, 96))
mask = torch.rand((8, 1, 224, 224))
aux = CutOut()(feature, mask)

In [ ]:
from Utils.functions import fix_all_seed
import random
import os
fix_all_seed(1)
train_data = sorted(os.listdir('proceeded_data/train/Image/'))
print(train_data[-1])
l_data = random.sample(train_data, int(len(train_data) * 0.04))
sorted(l_data)

In [ ]:
!python Datasets/process_resize.py

In [ ]:
%mkdir proceeded_data/images
%mkdir proceeded_data/labels

!mv proceeded_data/test/Image/* proceeded_data/images
!mv proceeded_data/train/Image/* proceeded_data/images
!mv proceeded_data/val/Image/* proceeded_data/images

!mv proceeded_data/test/Label/* proceeded_data/labels
!mv proceeded_data/train/Label/* proceeded_data/labels
!mv proceeded_data/val/Label/* proceeded_data/labels

In [ ]:
!rm -r proceeded_data/test/
!rm -r proceeded_data/train/
!rm -r proceeded_data/val/

In [ ]:
import os
img_n = os.listdir('proceeded_data/images')
label_n = os.listdir('proceeded_data/labels')
len(img_n), len(label_n)

In [ ]:
from Utils.functions import fix_all_seed
import random
import os

fix_all_seed(42)

n_sample = 739

imgs = sorted(os.listdir('proceeded_data/images'))
fold_1 = sorted(random.sample(imgs, n_sample))

imgs = [img for img in imgs if img not in fold_1]
fold_2 = sorted(random.sample(imgs, n_sample))

imgs = [img for img in imgs if img not in fold_2]
fold_3 = sorted(random.sample(imgs, n_sample))

imgs = [img for img in imgs if img not in fold_3]
fold_4 = sorted(random.sample(imgs, n_sample))

imgs = [img for img in imgs if img not in fold_4]
fold_5 = sorted(imgs)

print(f'Fold 1: {len(fold_1)} samples, Fold 2: {len(fold_2)} samples, Fold 3: {len(fold_3)} samples, Fold 4: {len(fold_4)} samples, Fold 5: {len(fold_5)} samples')

In [ ]:
folds = [fold_1, fold_2, fold_3, fold_4, fold_5]
for i in range(1, 6):
    with open(f'proceeded_data/fold{i}.txt', 'w') as f:
        for sample in folds[i - 1]:
            f.write(f'{sample}\n')

In [ ]:
from Datasets.create_dataset import *

supervised_ratio = 0.2
img_size = 224
path = 'proceeded_data/images'

folds = []
for i in range(5):
        fold = []
        with open(f'proceeded_data/fold{i + 1}.txt', 'r') as f:
            fold = [line.replace('\n', '') for line in f.readlines()]
            folds.append(fold)
    
datasets = {}
for i in range(5):
        train_data = []
        for j in range(5):
            if (j != i):
                train_data = [*train_data, *folds[j]]
        train_data = sorted(train_data)
        l_data = sorted(random.sample(train_data, int(len(train_data) * supervised_ratio)))
        u_data = sorted([sample for sample in train_data if sample not in l_data])
        l_dataset = SkinDataset(dataset=l_data, img_size=img_size, use_aug=True, data_path=path)
        u_dataset = SkinDataset(dataset=u_data, img_size=img_size, use_aug=True, data_path=path)
        
        val_data = sorted(folds[i])
        val_dataset = SkinDataset(dataset=val_data, img_size=img_size, use_aug=False, data_path=path)
        
        name = f'fold{i + 1}'
        datasets[name] = {
            'lb_dataset': l_dataset,
            'ulb_dataset': u_dataset,
            'val_dataset': val_dataset,
        }
        print(len(train_data))

In [ ]:
import os
import json
import torch
import random
import numpy as np
from torchvision import transforms
import albumentations as A
import pandas as pd
from Datasets.transform import *
from Datasets.unimatch import *
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt

path = './proceeded_data/'
img_size=224
train_data = os.listdir(path + 'images/')
print(len(train_data))
idx = random.randint(0, len(train_data))
# u_dataset = StrongWeakAugment(dataset=train_data, img_size=img_size, data_path=path)
# tensor_to_pil = transforms.ToPILImage()
# imgs = u_dataset[idx]
# display(tensor_to_pil(imgs['img_w']))
# display(tensor_to_pil(imgs['img_s']))
u_dataset = SemiDataset(dataset=train_data, img_size=img_size, use_aug=False, data_path=path)
tensor_to_pil = transforms.ToPILImage()
u_train_loader = torch.utils.data.DataLoader(u_dataset,
                                                batch_size=1,
                                                shuffle=True,
                                                num_workers=0,
                                                pin_memory=True,
                                                drop_last=False)
for idx, ((img_u_w, img_u_s1, img_u_s2, cutmix_box1, cutmix_box2),
            (img_u_w_mix, img_u_s1_mix, img_u_s2_mix, _, _)) in enumerate(zip(u_train_loader, u_train_loader)):
    #---------UniMatch-----------------------
    img_u_w = img_u_w.cuda()
    img_u_s1, img_u_s2 = img_u_s1.cuda(), img_u_s2.cuda()
    cutmix_box1, cutmix_box2 = cutmix_box1.cuda(), cutmix_box2.cuda()
    img_u_w_mix = img_u_w_mix.cuda()
    img_u_s1_mix, img_u_s2_mix = img_u_s1_mix.cuda(), img_u_s2_mix.cuda()

    img_u_s1[cutmix_box1.unsqueeze(1).expand(img_u_s1.shape) == 1] = \
                img_u_s1_mix[cutmix_box1.unsqueeze(1).expand(img_u_s1.shape) == 1]
    img_u_s2[cutmix_box2.unsqueeze(1).expand(img_u_s2.shape) == 1] = \
                img_u_s2_mix[cutmix_box2.unsqueeze(1).expand(img_u_s2.shape) == 1]
    
    print(img_u_s1[0].shape)
    print(img_u_s2[0].shape)
    display(transforms.ToPILImage()(img_u_s1[0]))
    display(transforms.ToPILImage()(img_u_s2[0]))
    plt.show()
    break

In [ ]:
from Utils.functions import fix_all_seed
from Datasets.create_dataset import *
fix_all_seed(1)
k = 5
supervised_ratio = 0.04
folds = []
for idx in range(1, 6):
        fold = []
        with open(f'proceeded_data/fold{idx}.txt', 'r') as f:
            fold = [line.replace('\n', '') for line in f.readlines()]
            folds.append(fold)
    
train_data = []
for j in range(5):
        if j != k - 1:
            train_data = [*train_data, *folds[j]]
train_data = sorted(train_data)
l_data = sorted(random.sample(train_data, int(len(train_data) * supervised_ratio)))
u_data = sorted([sample for sample in train_data if sample not in l_data])
print(l_data)

In [ ]:
!python -u ours.py --gpu 0 --seed 1 --exp ours --fold 1

In [ ]:
!python -u train_sup.py --gpu 0 --seed 1 --exp sup --fold 1
!python -u train_sup.py --gpu 0 --seed 1 --exp sup --fold 2
!python -u train_sup.py --gpu 1 --seed 1 --exp sup --fold 3
!python -u train_sup.py --gpu 1 --seed 1 --exp sup --fold 4
!python -u train_sup.py --gpu 1 --seed 1 --exp sup --fold 5

In [ ]:
!python -u fixmatch.py --gpu 0 --seed 1 --exp fixmatch --fold 1
!python -u fixmatch.py --gpu 0 --seed 1 --exp fixmatch --fold 2
!python -u fixmatch.py --gpu 1 --seed 1 --exp fixmatch --fold 3
!python -u fixmatch.py --gpu 1 --seed 1 --exp fixmatch --fold 4
!python -u fixmatch.py --gpu 1 --seed 1 --exp fixmatch --fold 5

In [ ]:
!python -u cps.py --gpu 0 --seed 1 --exp cps --fold 1

In [ ]:
import torch.nn.functional as F
import torch
import math
emb_u = torch.rand((8, 49, 768))
pseudo_prob_map = torch.rand((8, 1, 224, 224))
_, N, C = emb_u.size()
pseudo_prob_map = F.interpolate(pseudo_prob_map, size=(int(math.sqrt(N)), int(math.sqrt(N))),
                                                mode="bilinear", align_corners=False)

pseudo_prob_map = torch.flatten(pseudo_prob_map, start_dim=2, end_dim=-1)
print(F.normalize(pseudo_prob_map, dim=-1).shape)
print(F.normalize(emb_u, dim=1).shape)
cls_sim_matrix = torch.matmul(F.normalize(pseudo_prob_map, dim=-1), F.normalize(emb_u, dim=1))
channel_cls = torch.argmax(cls_sim_matrix, dim=1)
channel_cls.shape

In [ ]:
!python -u allspark.py --gpu 0 --seed 1 --exp allspark --fold 1

In [4]:
!python -u ours.py --gpu 0 --seed 1 --exp ours --fold 1

seed-----------all device 1
data:
  img_size: 224
  k_fold: 'No'
  name: isic2018
  supervised_ratio: 0.01
  test_folder: ./proceeded_data/
  train_aug: true
  train_folder: ./proceeded_data/
  val_folder: ./proceeded_data/
debug: false
device: cuda
fold: 1
log: false
model_adapt:
  adapt_method: false
  num_domains: false
model_encoder_id: 0
seed: 1
semi:
  conf_thresh: 0.85
swin:
  DEPTHS:
  - 2
  - 2
  - 18
  DROP_PATH_RATE: 0.2
  EMBED_DIM: 192
  NUM_HEADS:
  - 6
  - 12
  - 24
  WINDOW_SIZE: 7
  name: swin_large_patch4_window7_224_22k
test:
  batch_size: 5
  num_workers: 6
  only_test: false
  test_model_dir: ''
train:
  l_batchsize: 8
  num_epochs: 80
  num_iters: false
  num_workers: 0
  optimizer:
    adam:
      betas:
      - 0.9
      - 0.999
      lr: 1e-4
    adamw:
      betas:
      - 0.9
      - 0.999
      eps: 1e-8
      lr: 1e-4
      weight_decay: 0.05
    mode: adamw
  u_batchsize: 8

exp                 : ours
config_yml          : Configs/multi_train_local.yml
ada